In [1]:
import os, sys, ujson, json, pandas as pd, numpy as np, matplotlib.pyplot as plt, math
from sklearn.feature_selection import VarianceThreshold
from matplotlib.ticker import MultipleLocator, FormatStrFormatter

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from Feature.common import *
%matplotlib inline

In [6]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

# 筛选 feature

## 检查缺失值情况

In [ ]:
raw_df = cPickle.load(open('raw_feature.dat', 'rb'))

In [ ]:
# 用 20181016 的数据(有问题的数据)
保留输出不运行！！
for col in raw_df.columns:
    missing = len(raw_df) - np.count_nonzero( raw_df[col].isnull().values)
    mis_perc = 100 - float(missing) / len(raw_df) * 100
    
    if mis_perc > 50:
        print ("{miss} %\t {col}".format( col=col,miss=round(mis_perc,2)))

In [ ]:
# 用20180901的数据
print('missing ratio:\n')
for col in raw_df.columns:
    missing = np.count_nonzero( raw_df[col].isnull().values)
    mis_perc = float(missing) / len(raw_df) * 100
    
    if mis_perc :
        print ("{miss} %\t {col}".format(col=col,miss=round(mis_perc,2)))

>后一个cell 计算出的missing ratio 是基于 20181016日算的，前一个cell算的是20180901. 能明显看出，后者 push_ 相关feature缺省更为严重，这是因为系统bug

>剔除除了push以外的相关缺省严重的 feature。设 threshold = 50%。缺省一半以上的feature, 除了 push_相关 feature 都扔。

## discard unqiue columns

> 筛选 unqiue value 太多的，直接扔。筛选方差不大的，直接扔。最后 impute

In [ ]:
raw_df = cPickle.load(open('raw_feature.dat', 'rb'))

In [ ]:
# drop missing ratio 大的feature以后的数据：
unique_cols = []

for col in raw_df.columns:
    if isinstance( raw_df[col][0], dict) or isinstance(raw_df[col][0], list):
        continue
    
    values = raw_df[col]
    unique_values = [i for i in values if i != None and i != '']
    
    if unique_values:
        unique_perct = float(len(set(unique_values)))/len(unique_values)*100
    
        if unique_perct > 20:
            print ("{col}: \t\t{miss}%".format(col=col,miss=round(unique_perct,2)))

暂且保留：response_timestamp，想做 normalizing, scaling。 其他三个feature，丢～

## discard var ->0
> 筛选方差， 方差趋近于零的，丢～

In [ ]:
raw_df = cPickle.load(open('raw_feature.dat', 'rb'))

In [ ]:
# 找到所有 numerical cols
newdf = raw_df.select_dtypes(include=numerics)

for c in newdf.columns:
    var = np.var( newdf[c].values )
    
    if var < 3:
        print ("{var}\t, {col}".format(var = round(var, 2), col = c))

丢方差太小的： evergreen_confidence, no_of_videos (趋近于0)

'spam_word_count' 看起来怪怪得， 研究一下它的分布

## feature 相关性(working)

In [ ]:
dict_cols = []

for i in raw_df.columns:
    if isinstance( raw_df[i].)

raw_df.to_csv('excel_explore.csv', index = False)

In [ ]:
def split_feature_value_by_label(df, col):
    values_label_dict = defaultdict(list)
    
    for label, feature_value in zip( df['label'], df[col] ):
        values_label_dict[feature_value].append(label)

    return values_label_dict

def draw_label_log_distribution_by_feature( df, col, log = True, width = 1, perctg = True):
    values_label_dict = split_feature_value_by_label(df, col)
    
    positive_label_count = []
    negatie_label_count = []
    feature_value = []
    pos_perctg = []
    
    for x,y in values_label_dict.items():
        N = len(y)
        p = sum(y)
        n = N - p
        
        if log:
            positive_label_count.append( math.log(p+1) )
            negatie_label_count.append( math.log(n+1) ) 
            pos_perctg.append( math.log(p/N*100+1) )
            plt.ylabel('log count')
        else:
            positive_label_count.append( p )
            negatie_label_count.append( n )
            pos_perctg.append( float(p)/(p+n)*20000)
            plt.ylabel('count')
        feature_value.append(x)

    ax = plt.subplot(1,1,1)
    width = 1.0/len(pos_perctg)
    if perctg:
        plt.bar( np.arange(len(pos_perctg))+ width, \
        pos_perctg, 
        width = 0.2*width, 
        label = 'pos_%', 
        color = 'black')
    else:
        plt.bar( np.arange(len(positive_label_count)), \
                positive_label_count, \
                width = 0.2*width, \
                label = 'positive', \
                color = 'red')
        plt.bar( np.arange(len(positive_label_count))+ width, \
                negatie_label_count, 
                width = 0.2*width, 
                label = 'negative', 
                color = 'green')
        
    plt.title( str(col))
    plt.legend()
    plt.show()
    return feature_value, pos_perctg, positive_label_count, negatie_label_count

# def draw_label_perctg_by_feature(df, col, log = True):
#     ax = plt.subplot(1,1,1)
#     plt.plot( range(len(count)), count, '--*' )
#     ax.xaxis.set_major_locator( MultipleLocator(3))
#     plt.xlabel('app_language')
#     plt.ylabel('number of users')
#     plt.show()

draw_label_log_distribution_by_feature( raw_df, 'app_language', log = False, perctg=False)


> 以下feature在各个class 中分布近似一致，不是显著特征：

language

In [ ]:
for c in raw_df.columns[:5]:
    print(c)

# Encode label

> 将supervised keywords 啥的，两两配对，计算 similarity

raw feature 已经减到了 105

## 处理 feature 是 dict的

In [2]:
raw_df = cPickle.load(open('raw_feature.dat', 'rb'))

In [5]:
len(raw_df.columns), len(raw_df)

(105, 42852)

In [8]:
dict_features = []

for c in raw_df.columns:
    if c not in numerics and (isinstance(raw_df[c][0], list) or isinstance(raw_df[c][0], dict)):
        dict_features.append(c)

dict_features

[u'body_addons',
 u'category_v2_score',
 u'disgusting_scores',
 u'exp_scores',
 u'head_addons',
 u'image_mscv_scores',
 u'image_nsfw_scores',
 u'key_entities',
 u'key_entities_v2',
 u'key_entities_v2_hash',
 u'keywords',
 u'keywords_tag',
 u'keywords_v2',
 u'max_disgusting_scores',
 u'nl_category',
 u'nl_domain',
 u'nl_keywords',
 u'nl_subcategory',
 u'nl_supervised_keywords',
 u'nl_title_keywords',
 u'nl_topic',
 u'nl_topic2048',
 u'nl_topic256',
 u'nl_topic64',
 u'pictures',
 u'push_category',
 u'push_domain',
 u'push_keywords',
 u'push_subcategory',
 u'push_supervised_keywords',
 u'push_title_keywords',
 u'push_topic',
 u'push_topic2048',
 u'push_topic256',
 u'push_topic64',
 u'supervised_keywords',
 u'supervised_keywords_v2',
 u'supervised_keywords_v2_origin',
 u'thumbnail',
 u'title_keywords',
 u'topic',
 u'topic2048',
 u'topic256',
 u'topic64',
 u'topic_v2']

> 发现有些 feature 压根没有值， 丢～

body_addons

> 有些有用的dict，值展开，并且放个 others 类别

category_v2_score

In [19]:
values = []
for value in raw_df['category_v2_score']:
        if value:
            values.extend(value)
len(set(values)), set(values)

(23,
 {u'Accident',
  u'Art&Design',
  u'Auto',
  u'Business',
  u'Crime',
  u'Education',
  u'Fashion&Beauty',
  u'Food',
  u'Gaming',
  u'Health',
  u'Home&Garden',
  u'News_Entertainment',
  u'News_Politics',
  u'News_Sports',
  u'Others_Natural_Disaster',
  u'Others_Plane_Crash',
  u'Others_Terrorism',
  u'Pet_Animals',
  u'Prose',
  u'Religion',
  u'Science',
  u'Technology',
  u'Travel'})

# ToDoNext: 应该单独写一个函数，把所有dict value展开？如果user, context之间有对应关系的，就算各种similarity